In [8]:
import urllib3
from bs4 import BeautifulSoup
import pandas as pd
import random
import datetime
import dateparser
import re
import time
import math

In [9]:
time.time()


1600963353.930058

In [10]:
def manualScrapeNameAddress(html):
    # Why does this exist? Becasue:
    # <div class="clearfix">
    # <h2>Owner(s)</h3> ... </div>
    # Somehow, the html is formated wrong, so beautiful soup doesnt want to play nice with it. 
    htmlNoLines = html.replace('\n', '')
    htmlNoLines = htmlNoLines.replace('\r', '')
    htmlNoLines = htmlNoLines.replace('\t', '')
    
    groups = re.findall('<div class="clearfix">(\s+)(<h2>Owner(.*))<\/div>', htmlNoLines)
    ownerInfo = groups[0][1]
    try:
        lastDivLocation = ownerInfo.index('</div>')
        relavent = ownerInfo[:lastDivLocation]
        firstParagraph = relavent.index('<p>')
        nameAddress = relavent[firstParagraph:] #<p>Name</p> <p>Address</p>
        
        lilSoup = BeautifulSoup(nameAddress)
        paragraphs = lilSoup.find_all('p')
        return (paragraphs[0].text.strip(), paragraphs[1].text.strip())
    except Exception as e:
        print(html)
        print(e)
        return ('X', 'X')
    
    

In [11]:
class HomeSaleRecord:
    def __init__(self, parcelId, grantorName, granteeName, date, kind, price, included):
        self.parcelId = parcelId
        self.grantorName = grantorName
        self.granteeName = granteeName
        self.date = date
        self.kind = kind
        self.price = price
        self.included = included
        
    def __str__(self):
        firstLine = f'parcel: {self.parcelId}, date: {self.date}, grantor: {self.grantorName}, grantee: {self.granteeName}'
        secondLine = f'price: {self.price}, kind: {self.kind}, included: {self.included}'
        return firstLine + '\n' + secondLine
    
    def __repr__(self):
        firstLine = f'parcel: {self.parcelId}, date: {self.date}, grantor: {self.grantorName}, grantee: {self.granteeName}'
        secondLine = f'price: {self.price}, kind: {self.kind}, included: {self.included}'
        return firstLine + '\n' + secondLine
    
    def serialize(self):
        return {
            "Parcel": self.parcelId,
            "Date": self.date,
            "Buyer": self.granteeName,
            "Seller": self.grantorName,
            "Price": self.price,
            "Kind": self.kind,
            "Conveyance_Included": self.included
        }

class TaxAssessmentRecord:
    def __init__(self, parcelId, year, land, improvements, total):
        self.parcelId = parcelId
        self.year = year
        self.land = land
        self.improvements = improvements
        self.total = total
        
    def __str__(self):
        return f'parcel: {self.parcelId}, {self.year}, Land: {self.land}, Improvements: {self.improvements}, Total: {self.total}'
    
    def __repr__(self):
        return f'parcel: {self.parcelId}, {self.year}, Land: {self.land}, Improvements: {self.improvements}, Total: {self.total}'
    
    def serialize(self):
        return {
            "Parcel": self.parcelId,
            "Year": self.year,
            "Land_Assessment": self.land,
            "Improvement_Assessment": self.improvements,
            "Total_Assessment": self.total,
        }
    

def moneyStringToInt(moneyString):
    return int(moneyString.replace('$', '').replace(',', ''))

def moneyStringToFloat(moneyString):
    return float(moneyString.replace('$', '').replace(',', ''))
        
def getTaxAssessmentRecords(html_soup, parcelId):   
    tables = html_soup.find_all('table')
    propertyValueTable = list(filter(lambda t: 'Property Value' in t.text, tables))[0]
    propertyValueTableRows = propertyValueTable.find('tbody').find_all('tr')

    records = []
    for i in range(1, len(propertyValueTableRows)):
        tr = propertyValueTableRows[i]
        rowDatas = tr.find_all('td') 

        year = int(rowDatas[0].text.strip())
        
        land = moneyStringToInt(rowDatas[1].text.strip())
        improvments = moneyStringToInt(rowDatas[2].text.strip())
        total = moneyStringToInt(rowDatas[3].text.strip())
        record = TaxAssessmentRecord(parcelId, year, land, improvments, total)
        records.append(record)
    return records

def getMostRecentTaxAssessmentRecord(html_soup, parcelId):
    records = getTaxAssessmentRecords(html_soup, parcelId)
    return list(sorted(records, key=lambda taxRecord: taxRecord.year))[0]
           
    
def tryParseDate(dateString):
    try:
        return datetime.datetime.strptime(dateInput, '%m/%Y').date().isoformat()
    except:
        return '1900-01-01'
        
    
def getSalesRecordFromTable(tableElem, parcelId):
    # grantorName, granteeName, date, kind, price, included
    rows = tableElem.find_all('tr')
    grantor = rows[0].find('td').text.strip()
    grantee = rows[1].find('td').text.strip()
    
    thirdRowDatas = rows[2].find_all('td')
    dateInput = thirdRowDatas[0].text.strip()
    dateFormated = tryParseDate(dateInput)
    
    priceStr = thirdRowDatas[1].text.strip()
    price = moneyStringToFloat(priceStr)
    
    fourthRowDatas = rows[3].find_all('td')
    kind  = fourthRowDatas[0].text.strip()
    included = fourthRowDatas[1].text.strip()
    
    return HomeSaleRecord(parcelId, grantor, grantee, dateFormated, kind, price, included)
    

def getSalesRecords(html_soup, parcelId):
    table = html_soup.find('table')
    tableBodies = table.find_all('tbody')
    return [getSalesRecordFromTable(body, parcelId) for body in tableBodies]


def hasPropertyDetails(html_soup):
    if('No data was found for this property.' in html_soup.text):
        return False
    return True

def hasPropertySalesDetails(html_soup):
    if('No conveyance information is available online' in html_soup.text):
        return False
    return True

In [12]:
d = '9/2015'
ope = datetime.datetime.strptime(d, '%m/%Y')
dir(ope.date())
ope.date().isoformat()

'2015-09-01'

In [13]:
def getAllParcelIds():
    with open('all_parcel_ids.txt') as f:
        return f.readlines()
    return result

def getPropertyDetailsUrl(parcelId):
    return f'https://www.cityofmadison.com/assessor/property/propertydata.cfm?ParcelN={str(parcelId)}'

def getPropertySalesUrl(parcelId):
    #return f'https://www.cityofmadison.com/assessor/property/propertydata.cfm?ParcelN=0{str(parcelId)}&Type=S'
    return f'https://www.cityofmadison.com/assessor/property/additionalpropertydata.cfm?ParcelN={str(parcelId)}&Type=S'



In [15]:
# Read in unproccessed ids and processed ids
# Sample some ids from unproccessed ids
# Get 3 dataframes from those ids:
#     * All Sales
#     * All Tax Assessment Info
#     * ID --> Name
# Load the existing version of those dataframes
# Combine with the three new dataframes
# Save those 3 new, larger dataframes to memory, replacing the previos versions. 
# Save ids_unprocessed and ids_processed with the new versions of each.

def readfileAsSet(filename):
    with open(filename) as f:
        lines_clipped = [l.strip() for l in f.readlines()]
        return set(lines_clipped)
    
def writeSetToFile(filename, setToWrite):
    items = list(setToWrite)
    output = "\n".join(items)
    with open(filename, 'w') as f:
        f.write(output)

def processParcel(parcel_id, pool_manager):
    # Name, Most Recent TaxAssessment, List of Sales Info
    result = (None, None, None)
    
    details_url = getPropertyDetailsUrl(parcel_id)
    details_html = pool_manager.request('GET', details_url, preload_content=True).data.decode('utf-8')
    details_soup = BeautifulSoup(details_html)
    if(hasPropertyDetails(details_soup)):
        name, addr = manualScrapeNameAddress(details_html)
        taxAssessment = getMostRecentTaxAssessmentRecord(details_soup, parcel_id)
        result = (name, taxAssessment, None)
    
    
    sales_url = getPropertySalesUrl(parcel_id)
    sales_html = pool_manager.request('GET', sales_url, preload_content=True).data.decode('utf-8')
    sales_soup = BeautifulSoup(sales_html)
    if(hasPropertySalesDetails(sales_soup)):
        sales = getSalesRecords(sales_soup, parcel_id)
        itemA = result[0]
        itemB = result[1]
        result = (itemA, itemB, sales)

    return result

def processNParcels(n):
    http = urllib3.PoolManager()
    batch_size = 100
    num_batches = math.ceil(n / batch_size)
    
    for b in range(num_batches):
        ids_unprocessed = readfileAsSet('ids_unprocessed.txt')
        ids_processed = readfileAsSet('ids_processed.txt')
        df_names = pd.read_csv('parcel_names.csv')
        df_names.set_index("Parcel")
        df_assessments = pd.read_csv('parcel_assessments.csv')
        df_assessments.set_index("Parcel")
        df_sales = pd.read_csv('parcel_sales.csv')
        df_sales.set_index("Parcel")
        
        num_ids_to_sample = min(batch_size, len(ids_unprocessed))
        target_ids = random.sample(ids_unprocessed, num_ids_to_sample)
        
        for parcel_id in target_ids:
            print("|", end = "")
            owners_names, taxAssessment, homeSales = processParcel(parcel_id, http)
            
            # Names
            if(owners_names is not None):
                name_record = {"Parcel": parcel_id, "Owners": owners_names}
                df_names = df_names.append(name_record, ignore_index = True)
                
            # Tax Assessments
            if(taxAssessment is not None):
                df_assessments = df_assessments.append(taxAssessment.serialize(), ignore_index = True)
            
            # Home Sales
            if(homeSales is not None):
                for sale in homeSales:
                    df_sales = df_sales.append(sale.serialize(), ignore_index = True)
                    
        df_names.to_csv('parcel_names.csv', index=False)
        df_assessments.to_csv('parcel_assessments.csv', index=False)
        df_sales.to_csv('parcel_sales.csv', index=False)
        
        just_processed_ids = set(target_ids)
        new_processed_ids = ids_processed.union(just_processed_ids)
        new_unprocessed_ids = ids_unprocessed.difference(just_processed_ids)
        
        writeSetToFile('ids_processed.txt', new_processed_ids)
        writeSetToFile('ids_unprocessed.txt', new_unprocessed_ids)
        print('\nBATCH COMPLETE')
    
    
#df_names = pd.read_csv('parcel_names.csv')
#df_assessments = pd.read_csv('parcel_assessments.csv')
#df_sales = pd.read_csv('parcel_sales.csv')

before = time.time()
processNParcels(2500)
after = time.time()

print(after - before)

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
BATCH COMPLETE
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
BATCH COMPLETE
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
BATCH COMPLETE
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
BATCH COMPLETE
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
BATCH COMPLETE
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
BATCH COMPLETE
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
BATCH COMPLETE
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
BATCH COMPLETE
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

In [ ]:
df_test = None
#df_test = pd.DataFrame(columns = ["ID", "Name"])
df_test = pd.read_csv('test.csv')
df_test.set_index('ID')
df_test = df_test.append({"ID": "A1", "Name": "Ben"}, ignore_index = True)
df_test.to_csv('test.csv', index = False)
df_test

In [168]:
with open('allParcelIds.txt') as file_all:
    ids = ['0'+ ID.strip() for ID in file_all.readlines()]
    newContents = '\n'.join(ids)
    with open('all_parcel_ids.txt', 'w') as file_new:
        file_new.write(newContents)

In [163]:
tuple()

NameError: name 'date' is not defined

In [169]:
s = set()
s.add('one')
s.add('two')
writeSetToFile('boi.txt', s)